In [1]:
import sys
import importlib
sys.path.append('./qdarts')

from qdarts.experiment import Experiment
from qdarts.plotting import plot_polytopes

import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import src.utilities.utils  as u
import src.utilities.model_utils  as mu

import torch

In [2]:
path = "./datasets/sys_3_1__2/"
data, imgs, missing_folders, failed_folders = u.load_all_data(path, load_images=True)
data.keys()

Loading 25869 datapoints from ./datasets/sys_3_1__2/.
First file: ./datasets/sys_3_1__2/datapoint_00000\data.npz, Last file: ./datasets/sys_3_1__2/datapoint_25868\data.npz


dict_keys(['C_tilde_DD', 'C_DG', 'geometry', 'tc_meV', 'v_offset', 'x_voltage', 'y_voltage', 'alpha', 'E_c', 'cuts'])

In [5]:
imgs
context = u.create_context(data)
outputs = u.create_outputs(data) # TODO: make sure that you use E_c nor inv(C_tilde_DD)

In [6]:
imgs

[]